In [ ]:
import time
import numpy as np
import scipy.sparse as sps
from tqdm import tqdm
import matplotlib.pyplot as plt

class Environment:
    
    def __init__(self, n_nodes, n_features, features, n_steps_max_live_graph):
        matrix_shape = np.zeros((n_nodes,n_nodes)) 

        for j in tqdm(range(0,n_nodes)):
            p = np.zeros(n_nodes)
            parameters = np.random.binomial(1, 0.5, size=(n_nodes, n_features)) #parameters of features
            for i in range(0,n_nodes):
                p[i] = np.dot(features, parameters[i])
            matrix_shape[j] = p
 
        matrix = matrix_shape.T.copy()
        matrix /= np.sum(matrix, axis=0)
        np.fill_diagonal(matrix, 0.)
        self.matrix = matrix #matrix of probabilities of edges
        self.n_steps_max_live_graph = n_steps_max_live_graph
    
    def prog_cascade(self, n_steps_max_live_graph, n_nodes):
        
        realizations = np.zeros((n_nodes,n_nodes), dtype=int)
        activated_edges = self.matrix > np.random.rand(n_nodes,n_nodes)
        realizations[activated_edges] = 1
        return realizations

In [ ]:
def influence_cascade(matrix, n_steps_max_live_graph, temp_seeds):
    
    active_nodes = temp_seeds.copy()
    rand = np.random.rand

    newly_active_nodes = active_nodes.copy()
    t = 0
    while t < n_steps_max_live_graph and np.sum(newly_active_nodes) > 0:
        active_nodes_mask = np.array(newly_active_nodes, dtype=bool)
        p = matrix[active_nodes_mask]
        activated_edges = p > rand(p.shape[0], n_nodes)
        newly_active_nodes = 1 - active_nodes
        newly_active_nodes[~(np.any(activated_edges, axis=0))] = 0
        active_nodes = np.array(active_nodes + newly_active_nodes)
        t += 1

    return active_nodes


def monte_Carlo_Sampling(matrix, n_iterations_mc, temp_seeds):

    nodes = np.random.binomial(0, 1.0, size=(n_nodes))
    for _ in range(n_iterations_mc):
        nodes_live_graph = influence_cascade(matrix, n_steps_max_live_graph, temp_seeds)
        nodes = np.array(nodes + nodes_live_graph)
    nodes_by_iteration = nodes / n_iterations_mc
    return np.sum(nodes_by_iteration)


def greedy_algorithm(matrix, budget, n_steps_max_live_graph,  epsilon, delta, random=False, random_number=10):

    set_seeds = np.random.binomial(0, 1.0, size=(n_nodes))
    seeds_acquired = []
    prev_best = 0
    marginal_increase = -1
    if random == False: 
        for _ in range(budget):
            results_monte_carlo = []
            n_seeds = len(set_seeds.nonzero()[0])
            m_c_sampling_iterations = int((1 / (epsilon ** 2)) * np.log(n_seeds + 1) * np.log(1 / delta))
            if m_c_sampling_iterations == 0:
                m_c_sampling_iterations = 10


            for i in range(n_nodes):
                if set_seeds[i] == 0:
                    temp_seeds = set_seeds.copy()
                    temp_seeds[i] = 1
                    results_monte_carlo.append(monte_Carlo_Sampling(matrix, m_c_sampling_iterations,temp_seeds))
                else:
                    results_monte_carlo.append(0)

            best_arg = np.argmax(results_monte_carlo)
            marginal_increase = results_monte_carlo[best_arg] - prev_best

            if marginal_increase > 0:
                prev_best = results_monte_carlo[best_arg]
                set_seeds[best_arg] = 1
                seeds_acquired.append(best_arg)
                print("marginal increase : {:.2f}".format(results_monte_carlo[best_arg]))
                print("{} node acquired".format(best_arg) )
            else:
                print("marginal increase <= 0")
                break

        return set_seeds, seeds_acquired
    else: #Randomized research of new best seed
        for _ in range(budget):
            n_seeds = len(set_seeds.nonzero()[0])
            m_c_sampling_iterations = int((1 / (epsilon ** 2)) * np.log(n_seeds + 1) * np.log(1 / delta))
            if m_c_sampling_iterations == 0:
                m_c_sampling_iterations = 10

            marginal_increase = -1
            t = 0
            while t!=random_number and marginal_increase < 0:
                i = np.random.randint(0, matrix.shape[0])
                if set_seeds[i] == 0:
                    temp_seeds = set_seeds.copy()
                    temp_seeds[i] = 1
                    result_mc = monte_Carlo_Sampling(matrix, m_c_sampling_iterations,temp_seeds)
                    marginal_increase = result_mc - prev_best
                t+=1

            if marginal_increase > 0:
                prev_best = result_mc
                set_seeds[i] = 1
                seeds_acquired.append(i)
                print(marginal_increase)
                print("{} node acquired".format(i) )
            else:
                print("marginal increase <= 0")
                break

        return set_seeds, seeds_acquired



In [ ]:
class TS_Learner:
 
    def __init__(self, n_nodes):
        self.n = n_nodes
        self.a = np.ones((n_nodes, n_nodes))
        self.b = np.ones((n_nodes, n_nodes))
        
    def update(self, realizations):
        self.a = self.a + realizations
        self.b = self.b + np.ones((self.n, self.n)) - realizations
    
    def create_estimate_graph(self):
        matrix = np.random.beta(self.a, self.b, (self.n,self.n ))
        np.fill_diagonal(matrix, 0.)
        return matrix

In [ ]:
#Print Regret
n_nodes = 1000
n_features = 4
features = np.random.dirichlet(np.ones(n_features), size=1)

budget = 50
n_steps_max_live_graph = 5
epsilon = 0.3
delta = 0.2
random = True
random_number=50

differences_per_repetition = []
performance_per_repetition = []

T=50
Repetitions=50 


#Environment
env = Environment(n_nodes, n_features, features, n_steps_max_live_graph)
#TS Learner
ts = TS_Learner(n_nodes)

exp_time=time.time()
#cycle for of experiments
for i in range(T):
    
    #Update the learner #repetitions times
    for _ in tqdm(range(Repetitions)):
        realizations = env.prog_cascade(n_steps_max_live_graph, n_nodes)
        ts.update(realizations)
    #Estimated graph
    ts_matrix = ts.create_estimate_graph()
    #Find best Seeds from estimated graph
    start_time = time.time()
    best_seeds, seeds_acquired = greedy_algorithm(ts_matrix, budget, n_steps_max_live_graph,  epsilon, delta, random, random_number)
    print("Greedy n. {} computed in {:.2f} seconds".format(i, time.time()-start_time))
    print(seeds_acquired)
    #compute performance/reward of that seeds
    n_seeds = len(best_seeds.nonzero()[0])
    n_iterations_mc = int((1 / (epsilon ** 2)) * np.log(n_seeds + 1) * np.log(1 / delta))
    performance = monte_Carlo_Sampling(env.matrix, n_iterations_mc, best_seeds)
    performance_per_repetition.append(performance)
    
print("TS computed in {:.2f} seconds".format( time.time()-exp_time))
  
f1 = plt.figure(1)
plt.title("TS Influence Maximization\n(" + str(n_nodes) + " nodes | "  + str(budget)
              + " budget | " + str(epsilon)
              + " epsilon | " + str(delta) + " delta)")
plt.xlabel("Experiments")
plt.ylabel("Rewards")
plt.plot(performance_per_repetition, 'r')
plt.savefig("TS-Reward-{}n-r{}-e{}-d{}-s{}-S{}-b{}.png".format(n_nodes,random,epsilon,delta,n_steps_max_live_graph, S, budget), dpi=200)


best_seeds, seeds_acquired = greedy_algorithm(env.matrix, budget, n_steps_max_live_graph,  epsilon, delta, random, random_number)
optimal = monte_Carlo_Sampling(env.matrix, n_iterations_mc, best_seeds)

f1 = plt.figure(4)
plt.title("TS Influence Maximization\n(" + str(n_nodes) + " nodes | "  + str(budget)
              + " budget | " + str(epsilon)
              + " epsilon | " + str(delta) + " delta)")
plt.xlabel("Experiments")
plt.ylabel("Regret")
plt.plot(np.cumsum(optimal - performance_per_repetition), 'r')
plt.savefig("TS-Regret-{}n-r{}-e{}-d{}-s{}-S{}-b{}.png".format(n_nodes,random,epsilon,delta,n_steps_max_live_graph, S, budget), dpi=200)


plt.show()
